In [1]:
import pandas as pd

# 读取数据
data = pd.read_csv("data/1_dataset_fillna.csv")
data = data.drop(["M", "Anion", "CC"], axis=1)
data

,O,N,B,S,P,SSA,PV,RMIC,Dap,ID/IG,AML,PW,CD,Cs
0,7.15,3.52,0.00,0.0,0.0,1546.1,0.5753,0.939857,1.488390,1.029,1.00,1.6,0.10,85.0
1,8.12,3.79,3.66,0.0,0.0,1315.2,0.5116,0.931392,1.555961,1.038,1.00,1.6,0.10,114.0
2,21.55,3.29,0.00,0.0,0.0,1140.8,0.8600,0.209302,3.015428,0.820,2.50,1.8,0.10,90.2
3,21.55,3.29,0.00,0.0,0.0,1140.8,0.8600,0.209302,3.015428,0.820,2.50,1.8,0.25,86.3
4,21.55,3.29,0.00,0.0,0.0,1140.8,0.8600,0.209302,3.015428,0.820,2.50,1.8,0.50,79.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,4.68,5.02,0.00,0.0,0.0,166.6,0.1380,0.289855,3.313325,0.830,2.75,1.6,10.00,67.1
668,4.68,5.02,0.00,0.0,0.0,166.6,0.1380,0.289855,3.313325,0.830,2.75,1.6,20.00,63.1
669,4.68,5.02,0.00,0.0,0.0,166.6,0.1380,0.289855,3.313325,0.830,2.75,1.6,30.00,59.6
670,4.68,5.02,0.00,0.0,0.0,166.6,0.1380,0.289855,3.313325,0.830,2.75,1.6,40.00,59.3


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 按目标值分布按10个等级进行数据分割
data['target_class'] = pd.qcut(data['Cs'], q=10, labels=False)
X = data.drop('Cs', axis=1)
y = data['Cs']
stratify_column = data['target_class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=stratify_column)

# 数据标准化
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 将数据转换成DF以便后续使用
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

In [4]:
from tpot import TPOTRegressor

# 初始化TPOT回归模型
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42)

# 训练模型
tpot.fit(X_train, y_train)

# 在测试集上进行预测并评估模型
score = tpot.score(X_test, y_test)
print(f"模型在测试集上的R²评分: {score}")

# 导出最佳管道的Python代码
tpot.export('tpot_best_regression_pipeline.py')

# 预测结果
y_pred = tpot.predict(X_test)

C:\Users\41315\.conda\envs\automl-env\Lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -67.66722791859847

Generation 2 - Current best internal CV score: -67.19789399201875

Generation 3 - Current best internal CV score: -64.66272962478214

Generation 4 - Current best internal CV score: -64.66272962478214

Generation 5 - Current best internal CV score: -60.753076406272626

Best pipeline: XGBRegressor(StandardScaler(input_matrix), learning_rate=0.1, max_depth=6, min_child_weight=5, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=0.7500000000000001, verbosity=0)
模型在测试集上的R²评分: -69.72948672444048
